# Thesis project notebook 

##### Uncomment and run the cell below to automatically install the required libraries

In [33]:
# %pip install -r requirements.txt

Start by importing the required libraries

In [34]:
import json
import os
import random
from multiprocessing import Pool
import pandas as pd
import utils
import time

Let's load in the search terms we need to use

In [35]:
terms = pd.read_csv('search.tsv', sep='\t') # obtained from https://ahrefs.com/blog/top-youtube-searches/
terms.head(10)

,#,Keyword,Search Volume
0,1,bts,"16,723,304"
1,2,pewdiepie,"16,495,659"
2,3,asmr,"14,655,088"
3,4,billie eilish,"13,801,247"
4,5,baby shark,"12,110,100"
5,6,old town road,"10,456,524"
6,7,music,"10,232,134"
7,8,badabun,"10,188,997"
8,9,blackpink,"9,580,131"
9,10,fortnite,"9,117,342"


We can use these search terms to get YouTube video links which we can then use to gather comments. To save some time (and some precious API quota) we'll use the top 50 searched terms

This cell should not be run more than once a month. To make sure you never have to run it again, save the output to a file and load it for subsequent runs

In [36]:
# from get_video import get_videos
# half = terms.head(50)
# search_terms = half["Keyword"]
# urls = []

# for term in search_terms:
#     urls.append(get_videos(term))

# links = list(set(item for sublist in urls for item in sublist))

# with open("videos__.json", "w") as f:
#     json.dump(links, f)

In [37]:
with open("videos__.json", "r") as f:
    links = json.load(f)

print(f"This file contains <{len(links)}> links")

This file contains <911> links


This is a lot of links (and therefore a lot of data). However, taking into account the rate of analysis and the amount of comments per video, we realize that it's going to take a lot of time: many of these videos have around 20,000 comments minimum. Knowing that it takes this program around 30 seconds to analyze 100 comments, analyzing all comments would take days. 

Instead of waiting for 9 days, we can make a compromise: we can take a sample of 500 comments per video. 

Now that we have our list of links, we can start collecting data. Let's do the fast part first: collecting the like/dislike ratio and saving it somewhere convenient for later. 

As before, this cell should ideally be run once only as it takes a bit of time - around 15 minutes. If the notebook needs to be restarted, you should load the csv file instead.

In [38]:
# from get_video import get_likes
# start_time = time.time()

# with open("videos__.json", "r") as f:
#     links = json.load(f)
# df_list = []

# checkpoints = {int(len(links) * perc): perc * 100 for perc in [0.25, 0.5, 0.75, 0.9, 1]} # same as before

# for i, link in enumerate(links):
#     current_time = time.time()
#     elapsed_time = current_time - start_time
#     data = get_likes(link)
#     df_list.append(data)
#     time.sleep(1) # otherwise we will get an HTTP error
#     if i + 1 in checkpoints:
#             print(f"{checkpoints[i+1] :.0f}% of the videos have been analyzed\nTime elapsed: {elapsed_time :.2f} seconds")  
     
# df = pd.DataFrame(df_list)
# utils.save_analysis(df, "top50_ratio")

Let's look at the first 10 entries

In [39]:
likes_df = pd.read_csv("analysis_top50_ratio.csv", index_col = 0)
likes_df = likes_df.reset_index()
likes_df.head(10)

,URL,likes,dislikes
0,https://www.youtube.com/watch?v=feLcMaiClOw,312761,19093
1,https://www.youtube.com/watch?v=tASwv-tkMlc,35026,165
2,https://www.youtube.com/watch?v=3U2dNKBM28o,126427,6010
3,https://www.youtube.com/watch?v=1cPDfXU95Xw,23783,18
4,https://www.youtube.com/watch?v=pptIU_ZLlOo,4600,7
5,https://www.youtube.com/watch?v=ClQ-ymoXJZc,1079676,8874
6,https://www.youtube.com/watch?v=8EJ3zbKTWQ8,11371069,1420624
7,https://www.youtube.com/watch?v=t99KH0TR-J4,1435738,33092
8,https://www.youtube.com/watch?v=DovdIspaqmw,146007,5324
9,https://www.youtube.com/watch?v=DQQRjFzB8gY,870202,49922


This is good data, but it is not complete. We are still missing the ratio of likes to dislikes. Let's compute that now!

In [40]:
likes, dislikes = likes_df['likes'], likes_df['dislikes']

ratios = []

for like, dislike in zip(likes, dislikes):
    if like == 0 or dislike == 0:
        ratio = 0
        ratios.append(ratio)
    else:
        ratio = round(like / dislike, 3)
        ratios.append(ratio)

ratios = pd.Series(ratios)
ratios.head(10)

0      16.381
1     212.279
2      21.036
3    1321.278
4     657.143
5     121.667
6       8.004
7      43.386
8      27.424
9      17.431
dtype: float64

Let's add this list to our dataframe and compute the mean ratio

In [41]:
likes_df['ratio'] = ratios.astype('float64') 
mean_ratio = ratios.mean()
print(f"Mean like/dislike ratio = {mean_ratio :.2f}")
likes_df.head(10)

Mean like/dislike ratio = 123.55


,URL,likes,dislikes,ratio
0,https://www.youtube.com/watch?v=feLcMaiClOw,312761,19093,16.381
1,https://www.youtube.com/watch?v=tASwv-tkMlc,35026,165,212.279
2,https://www.youtube.com/watch?v=3U2dNKBM28o,126427,6010,21.036
3,https://www.youtube.com/watch?v=1cPDfXU95Xw,23783,18,1321.278
4,https://www.youtube.com/watch?v=pptIU_ZLlOo,4600,7,657.143
5,https://www.youtube.com/watch?v=ClQ-ymoXJZc,1079676,8874,121.667
6,https://www.youtube.com/watch?v=8EJ3zbKTWQ8,11371069,1420624,8.004
7,https://www.youtube.com/watch?v=t99KH0TR-J4,1435738,33092,43.386
8,https://www.youtube.com/watch?v=DovdIspaqmw,146007,5324,27.424
9,https://www.youtube.com/watch?v=DQQRjFzB8gY,870202,49922,17.431


We will use the data in this dataframe as the baseline for the sentiment analysis.

This process is going to take a long time to complete, even with multiprocessing. 

Let's start by collecting the comments.
The results will be saved to a file, so there's no need to run this cell more than once if everything goes well.

In [42]:
# from get_video import get_id, get_comments
# import googleapiclient.discovery

# urls = list(likes_df["URL"])

# target_dir = f"{os.getcwd()}/Data"
# if not os.path.exists(target_dir):
#     os.mkdir(target_dir)

# for url in urls:
#     video_id = get_id(url)
#     try:
#         comments = get_comments(url)
#     except googleapiclient.errors.HttpError:
#         continue
#     utils.move_dir(f"comments_{video_id}.json", target_dir)

There is a problem: some of the files have very few comments, and we need at least 500. So, we will have to remove all files with less than 500 comments

In [43]:
# target_dir = f"{os.getcwd()}\\Data"
# trash = f"{os.getcwd()}\\Trash"

# checkpoints = {int(len(links) * perc): perc * 100 for perc in [0.25, 0.5, 0.75, 0.9, 1]} # same as before

# if not os.path.exists(trash):
#     os.mkdir(trash)

# data = os.listdir(target_dir)

# start =  time.time()

# for i, file in enumerate(data):
#     current_time = time.time()
#     elapsed_time = current_time - start
#     filepath = os.path.join(target_dir, file)
#     content = load_comments(filepath)
#     if len(content) < 500:
#         utils.move_dir(filename = file, destination = trash, source = target_dir)
#     if i + 1 in checkpoints:
#         print(f"{checkpoints[i+1] :.0f}% of the videos have been analyzed\nTime elapsed: {elapsed_time :.2f} seconds") 
# trash_list = os.listdir(trash)

# print(f"<{len(trash_list)}> files moved")

Quick sanity check:

In [44]:
# target_dir = f"{os.getcwd()}\\Data"

# data = os.listdir(target_dir)
# print(f"This directory contains <{len(data)}> files")
# for file in data:
#     filepath = os.path.join(target_dir, file)
#     content = load_comments(filepath)
#     print(len(content))

Great! We have over 500 videos left, each with at least 500 comments. All that's left to do is run the sentiment analysis on each video. This is going to take a lot of time, which means we'll have to do it in batches. 100 videos per batch seems feasible, so let's do that. 

In [45]:
# %run split_folder__.py "F:\Desktop\Scuola\Uni\Y3\S2\Thesis\Project\Main project\GetYoutubeComments\Data" 
# #Obtained from https://gist.github.com/zupo/5849843

Another sanity check:

In [46]:
# target_dir = f"{os.getcwd()}\\Data\\Batch_1"
# data = os.listdir(target_dir)
# print(len(data))

Now we can finally start the analysis

In [47]:
# from get_video import load_comments

# batch_5_5 = f"{os.getcwd()}\\Data\\Batch_5_5"
# data_5_5 = os.listdir(batch_5_5)

# os.chdir(batch_5_5)

# for file in data_5_5:
#     if file.startswith("analysis_"): 
#         continue
#     else:
#         start = time.time()
#         filename = f"{batch_5_5}\\{file}"
#         content = load_comments(filename)

#         with Pool() as p:
#             tr_comment = p.map(utils.translate, content)

#         bert_scores = utils.bert_classifier(content)

#         df_list = []
#         scores_list = []

#         with Pool() as p:
#             analysis1 = p.map(utils.vader_classifier, tr_comment)
#             analysis2 = p.map(utils.textblob_classifier, tr_comment)
#             scores_list.append(analysis1)
#             scores_list.append(analysis2)

#         for c, v_score, t_score in zip(content, scores_list[0], scores_list[1]):
#             df_dict = {
#                 "Comment": c,
#                 "Vader score": v_score,
#                 "TextBlob score": t_score,
#             }
#             df_list.append(df_dict)

#         df = pd.DataFrame(df_list)

#         series_bert = pd.Series(bert_scores)
#         df["BERT score"] = series_bert

#         vader_labels = utils.generate_labels(df["Vader score"], "vader")
#         blob_labels = utils.generate_labels(df["TextBlob score"], "blob")
#         bert_labels = utils.generate_labels(df["BERT score"], "bert")

#         vader_labels_series = pd.Series(vader_labels)
#         blob_labels_series = pd.Series(blob_labels)
#         bert_labels_series = pd.Series(bert_labels)

#         df["Vader label"] = vader_labels_series
#         df["TextBlob label"] = blob_labels_series
#         df["BERT label"] = bert_labels_series

#         df = df[
#             [
#                 "Comment",
#                 "Vader score",
#                 "Vader label",
#                 "TextBlob score",
#                 "TextBlob label",
#                 "BERT score",
#                 "BERT label",
#             ]
#         ]

#         end = time.time()
#         time_taken = end - start
#         print(f"Processed file <{file}>\nTime taken: <{time_taken}> seconds")
#         utils.save_analysis(df, file)

# os.chdir("F:\\Desktop\\Scuola\\Uni\\Y3\\S2\\Thesis\\Project\\Main project\\GetYoutubeComments") # Reset cwd for next batch

In [48]:
from get_video import load_comments

batch_6 = f"{os.getcwd()}\\Data\\Batch_6"
data_6 = os.listdir(batch_6)

os.chdir(batch_6)

for file in data_6:
    if file.startswith("analysis_"): 
        continue
    else:
        start = time.time()
        filename = f"{batch_6}\\{file}"
        content = load_comments(filename)

        with Pool() as p:
            tr_comment = p.map(utils.translate, content)

        bert_scores = utils.bert_classifier(content)

        df_list = []
        scores_list = []

        with Pool() as p:
            analysis1 = p.map(utils.vader_classifier, tr_comment)
            analysis2 = p.map(utils.textblob_classifier, tr_comment)
            scores_list.append(analysis1)
            scores_list.append(analysis2)

        for c, v_score, t_score in zip(content, scores_list[0], scores_list[1]):
            df_dict = {
                "Comment": c,
                "Vader score": v_score,
                "TextBlob score": t_score,
            }
            df_list.append(df_dict)

        df = pd.DataFrame(df_list)

        series_bert = pd.Series(bert_scores)
        df["BERT score"] = series_bert

        vader_labels = utils.generate_labels(df["Vader score"], "vader")
        blob_labels = utils.generate_labels(df["TextBlob score"], "blob")
        bert_labels = utils.generate_labels(df["BERT score"], "bert")

        vader_labels_series = pd.Series(vader_labels)
        blob_labels_series = pd.Series(blob_labels)
        bert_labels_series = pd.Series(bert_labels)

        df["Vader label"] = vader_labels_series
        df["TextBlob label"] = blob_labels_series
        df["BERT label"] = bert_labels_series

        df = df[
            [
                "Comment",
                "Vader score",
                "Vader label",
                "TextBlob score",
                "TextBlob label",
                "BERT score",
                "BERT label",
            ]
        ]

        end = time.time()
        time_taken = end - start
        print(f"Processed file <{file}>\nTime taken: <{time_taken}> seconds")
        utils.save_analysis(df, file)

os.chdir("F:\\Desktop\\Scuola\\Uni\\Y3\\S2\\Thesis\\Project\\Main project\\GetYoutubeComments") # Reset cwd for next batch